## 1. Quá trình dự báo trên mô hình GRU

Thêm thư viện

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.layers import GRUV2 as GRU

Nạp dữ liệu từ file csv

In [2]:

start = '01/06/2020'
end = '01/06/2022'
pre_day = 30
stock="bitcoin"
# df = investpy.get_crypto_historical_data(crypto=stock ,from_date=start, to_date=end)
df=pd.read_csv('bitcoin_train_7_3.csv')
df = pd.DataFrame(df)
df

,Date,Open,High,Low,Close,Volume,Currency
0,2020-06-01,9.4545,10.3018,9.4297,10.1893,796676,USD
1,2020-06-02,10.1907,10.2073,9.3473,9.5276,1058556,USD
2,2020-06-03,9.5270,9.6672,9.4217,9.6672,558116,USD
3,2020-06-04,9.6672,9.8644,9.4909,9.7944,593664,USD
4,2020-06-05,9.7940,9.8461,9.6287,9.6312,527054,USD
...,...,...,...,...,...,...,...
507,2021-10-21,66.0046,66.6169,62.0921,62.2102,105380,USD
508,2021-10-22,62.2008,63.6997,60.0557,60.6903,75688,USD
509,2021-10-23,60.6932,61.7287,59.7383,61.3125,37046,USD
510,2021-10-24,61.3082,61.4699,59.5237,60.8665,42674,USD


Chuẩn hóa dữ liệu

In [3]:
df['H-L'] = df['High'] - df['Low']
df['O-C'] = df['Open'] - df['Close']
ma_1 = 7
ma_2 = 14
ma_3 = 21
# Đường Simple Moving Average (SMA)
# là đường trung bình động đơn giản được tính bằng trung bình cộng các mức giá đóng cửa trong một khoảng thời gian giao dịch nhất định.
# Vd MA_7 = Khoảng thời gian (7 ngày về sau tính cả ngày đang tính)
df[f'SMA_{ma_1}'] = df['Open'].rolling(window=ma_1).mean()
df[f'SMA_{ma_2}'] = df['Open'].rolling(window=ma_2).mean()
df[f'SMA_{ma_3}'] = df['Open'].rolling(window=ma_3).mean()

df[f'SD_{ma_1}'] = df['Open'].rolling(window=ma_1).std()
df[f'SD_{ma_3}'] = df['Open'].rolling(window=ma_3).std()
df.dropna(inplace=True)

df.to_csv(f"{stock}7_3_standardized.csv")
print(df)

           Date     Open     High      Low    Close  Volume Currency     H-L  \
20   2020-06-21   9.3588   9.4112   9.2885   9.2964  335994      USD  0.1227   
21   2020-06-22   9.2965   9.7519   9.2858   9.6837  523319      USD  0.4661   
22   2020-06-23   9.6839   9.7156   9.5818   9.6246  375494      USD  0.1338   
23   2020-06-24   9.6246   9.6674   9.2230   9.3020  515777      USD  0.4444   
24   2020-06-25   9.3008   9.3273   9.0226   9.2475  524453      USD  0.3047   
..          ...      ...      ...      ...      ...     ...      ...     ...   
507  2021-10-21  66.0046  66.6169  62.0921  62.2102  105380      USD  4.5248   
508  2021-10-22  62.2008  63.6997  60.0557  60.6903   75688      USD  3.6440   
509  2021-10-23  60.6932  61.7287  59.7383  61.3125   37046      USD  1.9904   
510  2021-10-24  61.3082  61.4699  59.5237  60.8665   42674      USD  1.9462   
511  2021-10-25  60.8643  63.6948  60.6558  63.0670   51061      USD  3.0390   

        O-C      SMA_7     SMA_14     S

Scale Miền giá trị lại

In [4]:
# Process Data
pre_day = 30
scala_x = MinMaxScaler()
scala_y = MinMaxScaler()
cols_x = ['H-L', 'O-C', f'SMA_{ma_1}', f'SMA_{ma_2}', f'SMA_{ma_3}', f'SD_{ma_1}', f'SD_{ma_3}']
cols_y = ['Open']
scaled_data_x = scala_x.fit_transform(df[cols_x].values.reshape(-1, len(cols_x)))
scaled_data_y = scala_y.fit_transform(df[cols_y].values.reshape(-1, len(cols_y)))
# scaled_data_x
scaled_data_y

array([[0.00614778],
       [0.00505472],
       [0.01185167],
       [0.01081125],
       [0.00513017],
       [0.00419502],
       [0.0026686 ],
       [0.        ],
       [0.00200715],
       [0.003116  ],
       [0.00223699],
       [0.00388622],
       [0.00134395],
       [0.00102989],
       [0.00220892],
       [0.00126324],
       [0.00579688],
       [0.00434941],
       [0.00739874],
       [0.00404588],
       [0.00486699],
       [0.00396167],
       [0.0051249 ],
       [0.00411782],
       [0.00430204],
       [0.00333882],
       [0.00222822],
       [0.00258614],
       [0.00283703],
       [0.00350374],
       [0.00270018],
       [0.00664957],
       [0.00886375],
       [0.0103656 ],
       [0.00944449],
       [0.01195869],
       [0.01621687],
       [0.03426018],
       [0.03333556],
       [0.03679894],
       [0.03663578],
       [0.04078868],
       [0.04902432],
       [0.03611644],
       [0.03886926],
       [0.038185  ],
       [0.04783828],
       [0.048

In [5]:
x_total = []
y_total = []

for i in range(pre_day, len(df)):
    x_total.append(scaled_data_x[i-pre_day:i])
    y_total.append(scaled_data_y[i])
x_total=np.array(x_total)
y_total=np.array(y_total)

Tách tập dữ liệu thành train và test

In [6]:
# Build Model
model = Sequential()

model.add(GRU(units=60, return_sequences=True, input_shape=(x_total.shape[1], x_total.shape[2])))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60, return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=60))
model.add(Dropout(0.2))
model.add(Dense(units=len(cols_y)))

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_total, y_total, epochs=60, steps_per_epoch=40, use_multiprocessing=True)
model.save(f"{stock}73.h5")
print("Done Training Model")

Epoch 1/60
40/40 [==============================] - 14s 44ms/step - loss: 0.0423
Epoch 2/60
40/40 [==============================] - 2s 44ms/step - loss: 0.0109
Epoch 3/60
40/40 [==============================] - 2s 48ms/step - loss: 0.0092
Epoch 4/60
40/40 [==============================] - 2s 47ms/step - loss: 0.0095
Epoch 5/60
40/40 [==============================] - 2s 45ms/step - loss: 0.0093
Epoch 6/60
40/40 [==============================] - 2s 46ms/step - loss: 0.0064
Epoch 7/60
40/40 [==============================] - 2s 45ms/step - loss: 0.0092
Epoch 8/60
40/40 [==============================] - 2s 48ms/step - loss: 0.0057
Epoch 9/60
40/40 [==============================] - 2s 55ms/step - loss: 0.0049
Epoch 10/60
40/40 [==============================] - 2s 56ms/step - loss: 0.0047
Epoch 11/60
40/40 [==============================] - 2s 57ms/step - loss: 0.0046
Epoch 12/60
40/40 [==============================] - 2s 54ms/step - loss: 0.0047
Epoch 13/60
40/40 [=================